<a href="https://colab.research.google.com/github/HarryLatThinkCerca/ExploreData/blob/main/metals-model1-classification_version3_from_jenny_Mar27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model1 Classification Version 3
- updated on 3/27 by Jenny Song
<br></br>
- (*new) used all data from 6 groups
<br></br>
- (*new) pre-computed features to speed up training time
<br></br>
- (*new) embed tensorboard to kaggle notebook 
<br></br>
- validation evaluation for each rurbic item at each epoch: 
    - confusion matrix 
    - agreement accuracy
    - (*new) adjacent accuracy
<br></br>  
- classification
    - use Cross Entropy loss
    - top-1 accuracy metric
<br></br>     
- model
    - bert
    - (*new) roberta
    - (*new) t5

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/thinkcerca'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/thinkcerca/group_6_final_writing_for_lesson_25979.xlsx - Result 1.csv
/kaggle/input/thinkcerca/Group_2_8492_rubric_scores_for_lesson_id_24096_fixed.xlsx - Result 1.csv
/kaggle/input/thinkcerca/group_3_final_writing_for_lesson_20846.xlsx - Result 1.csv
/kaggle/input/thinkcerca/Group_5_all_rubric_scores_group_5_lesson_25494_7176_scores.xlsx - Result 1.csv
/kaggle/input/thinkcerca/group_4_final_writing_for_lesson_24713.xlsx - Result 1.csv
/kaggle/input/thinkcerca/group_2_final_writing_lesson_24096.xlsx - Result 1.csv
/kaggle/input/thinkcerca/group_1_final_writing_for_lesson_26025.xlsx - Result 1.csv
/kaggle/input/thinkcerca/group_5_final_writing_for_lesson_25494.xlsx - Result 1.csv
/kaggle/input/thinkcerca/Group_6_25979_rubric_scores_8091_total_scores.xlsx - Result 1.csv
/kaggle/input/thinkcerca/Group_4_lesson_24713_8636_rubric_scores.xlsx - Result 1.csv
/kaggle/input/thinkcerca/Group_3_all_10182_rubric_scores_lesson20846.xlsx - Result 1.csv
/kaggle/input/thinkcerca/Group_1_

# Load dataframes

In [ ]:
all_writings, all_rubrics = [], []
for dirname, _, filenames in os.walk('/kaggle/input/thinkcerca'):
    for filename in filenames:
        if 'writing' in filename:
            all_writings.append(os.path.join(dirname, filename))
        else:
            all_rubrics.append(os.path.join(dirname, filename))

In [ ]:
final_writing_list = []
for writing_file in all_writings:
    df = pd.read_csv(writing_file, skipinitialspace=True)
    if "group_2" in writing_file:
        df.rename({'final_writing': 'sr_final_writing'}, axis=1, inplace=True)
    final_writing = df[['sa_id','sr_final_writing']].dropna()
    final_writing_list.append(final_writing)
group_final_writing = pd.concat(final_writing_list).reset_index()

final_rubric_list = []
for rubric_file in all_rubrics:
    df = pd.read_csv(rubric_file, skipinitialspace=True)
    final_rubric = df[['sa_id','a_rubric_id','rc_id','ai_score']].dropna()
    final_rubric_list.append(final_rubric)
group_final_rubric = pd.concat(final_rubric_list).reset_index()

In [ ]:
group_final_rubric

,index,sa_id,a_rubric_id,rc_id,ai_score
0,12,9186611,9,49,2.0
1,13,9186611,9,51,3.0
2,14,9186611,9,53,2.0
3,15,9186611,9,55,1.0
4,16,9186611,9,57,3.0
...,...,...,...,...,...
57317,14752,16555289,252,1145,3.0
57318,14753,16555289,252,1146,3.0
57319,14754,16555289,252,1147,3.0
57320,14755,16555289,252,1148,4.0


# Data filtering

In [ ]:
rubric_id_to_rc_id = {252: 1143, 233: 1018, 234: 1025}

import collections
data = collections.defaultdict(dict)
for _,row in group_final_writing.iterrows():
    sa_id = row['sa_id']
    # filter out final writing that have fewer than 30 characters
    if len(row['sr_final_writing']) > 30:
        data[sa_id]['final_writing'] = row['sr_final_writing']
        data[sa_id]['scores'] = [-1]*7

for _,row in group_final_rubric.iterrows():
    sa_id = row['sa_id']
    # filter out no writing and rubric id 9 because rubric item difference
    if (not sa_id in data) or (row['a_rubric_id'] == 9):
        continue
    rc_id_base = rubric_id_to_rc_id[row['a_rubric_id']]
    
    index = int(row['rc_id'])-rc_id_base
    data[sa_id]['scores'][index]=row['ai_score']

to_drop = []
for key, val in data.items():
    score_cur = val['scores']
    score_cur_array = np.array(score_cur)

    if -1 in score_cur:
        to_drop.append(key)
        continue

for k in to_drop:
    del data[k]

data_final = [v for v in data.values()]

# Dataset

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl
from transformers import BertTokenizer, BertModel
from torchmetrics import ConfusionMatrix



class WritingDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data
        

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]['final_writing'], torch.tensor(self.data[idx]['scores'])
        

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import RobertaTokenizer, RobertaModel
from transformers import T5Tokenizer, T5EncoderModel

NUM_CATEGORIES = 6
NUM_RUBRICS = 7
tokenizers = ["roberta-base", "bert-base-uncased", "t5-large"]
CURR_MODEL = "t5-large"

class Trainer(pl.LightningModule):
    def __init__(self):
        super().__init__()
        if CURR_MODEL == "t5-large":
            self.cls = nn.Sequential(
                nn.Linear(1024, 512),
                nn.ReLU(),
                nn.Linear(512, 250),
                nn.ReLU(),
                nn.Linear(250, NUM_CATEGORIES * NUM_RUBRICS)
            )
        else:
            self.cls = nn.Sequential(
                nn.Linear(768, 500),
                nn.ReLU(),
                nn.Linear(500, 250),
                nn.ReLU(),
                nn.Linear(250, NUM_CATEGORIES * NUM_RUBRICS)
            )
        self.tokenizer = T5Tokenizer.from_pretrained("t5-large")
        self.bert = T5EncoderModel.from_pretrained("t5-large").cuda()
        self.nlp = self.text_feature_extraction
        
    def text_feature_extraction(self, text):
        with torch.no_grad():
            if CURR_MODEL == "t5-large":
                input_ids = self.tokenizer(text, return_tensors="pt",padding=True,truncation=True,max_length=512).input_ids.cuda()
                outputs = self.bert(input_ids=input_ids)
            else:
                inputs = self.tokenizer(text, return_tensors="pt",padding=True,truncation=True,max_length=512)
                for key in inputs:
                    inputs[key] = inputs[key].cuda()
                outputs = self.bert(**inputs)
            # outputs.last_hidden_state B x H x 768
            return outputs.last_hidden_state.mean(dim=1) # B x 768, #B x 512 for T5
        
    def forward(self, text):
        text_length = [len(t) for t in text] # B
        text_feature = self.nlp(text) # B x 768
        return self.cls(text_feature)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
  
    
    def training_step(self, train_batch, batch_idx):
        sentence, scores = train_batch
        logits = self.forward(sentence).reshape(-1, NUM_CATEGORIES, NUM_RUBRICS) # B x 6 x 8
        loss = F.cross_entropy(logits, scores.long())
        
        self.log('train_ce_loss', loss, batch_size=len(sentence))
        preds = logits.argmax(dim=1)
        self.log('train_acc', (preds == scores).float().mean(), batch_size=len(sentence))
        return loss

    def validation_step(self, val_batch, batch_idx):
        sentence, scores = val_batch
        logits = self.forward(sentence).reshape(-1, NUM_CATEGORIES, NUM_RUBRICS) # B x 6 x 8
        
        loss = F.cross_entropy(logits, scores.long())
        
        self.log('val_ce_loss', loss, batch_size=len(sentence))
        preds = logits.argmax(dim=1)
        self.log('val_acc', (preds == scores).float().mean(), batch_size=len(sentence))
        
        return {"scores": scores, "pred": preds}
    
    def validation_epoch_end(self, validation_step_outputs):
        scores_list = []
        prediction_list = []
        for step in validation_step_outputs:
            s = step["scores"]
            scores_list.append(s)
            p = step["pred"]
            prediction_list.append(p)
        scores = torch.cat(scores_list)
        prediction = torch.cat(prediction_list)
      
            
        scores_int = scores.int()
        prediction_int = prediction.int()
        
        prediction_int[prediction_int<1]=0
        prediction_int[prediction_int>5]=5
        
        
        for i in range(NUM_RUBRICS):
            confusion_matrix = ConfusionMatrix(num_classes=6)(prediction_int[:,i].cpu(), scores_int[:,i].cpu())
#             print(f"confusion matrix {i}", confusion_matrix.numpy())
            
            df_cm = pd.DataFrame(confusion_matrix.numpy(), index = range(6), columns=range(6))
            plt.figure(figsize = (10,7))
            fig_ = sns.heatmap(df_cm, annot=True, cmap='Blues').get_figure()
            plt.close(fig_)

            self.logger.experiment.add_figure(f"Confusion matrix {i}", fig_, self.current_epoch)
        
        
        acc_per_rubric_item = torch.mean((prediction == scores).float(), dim=0)
        

        
        self.log('val_Claim_acc', acc_per_rubric_item[0])
        self.log('val_Reasons_acc', acc_per_rubric_item[1])
        self.log('val_Evidence_acc', acc_per_rubric_item[2])
        self.log('val_Reasoning_acc', acc_per_rubric_item[3])
        self.log('val_Organization_acc', acc_per_rubric_item[4])
        self.log('val_Audience_acc', acc_per_rubric_item[5])
        self.log('val_English_acc', acc_per_rubric_item[6])

        

In [ ]:
pl.utilities.seed.seed_everything(seed=1)
        
# # data
dataset = WritingDataset(data_final)
# mnist_train, mnist_val = random_split(dataset, [int(len(dataset)*0.8), len(dataset)-int(len(dataset)*0.8)])

# train_loader = DataLoader(mnist_train, batch_size=32, shuffle=True)
# val_loader = DataLoader(mnist_val, batch_size=64, shuffle=False)

# # model
model = Trainer()

# checkpoint_callback = pl.callbacks.model_checkpoint.ModelCheckpoint(
#     monitor="val_acc",
#     dirpath="checkpoints/",
#     filename="cls-all-data-3-layers-{epoch:02d}-{val_acc:.2f}",
#     save_top_k=2,
#     mode="max",
# )

# # training
# trainer = pl.Trainer(gpus=1, precision=16, max_epochs=20,callbacks=[checkpoint_callback])
# trainer.fit(model, train_loader, val_loader)
    


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.75G [00:00<?, ?B/s]

Some weights of the model checkpoint at t5-large were not used when initializing T5EncoderModel: ['decoder.block.1.layer.2.layer_norm.weight', 'decoder.block.19.layer.0.SelfAttention.k.weight', 'decoder.block.8.layer.0.SelfAttention.v.weight', 'decoder.block.21.layer.1.layer_norm.weight', 'decoder.block.22.layer.0.SelfAttention.k.weight', 'decoder.block.1.layer.1.EncDecAttention.k.weight', 'decoder.block.3.layer.0.SelfAttention.o.weight', 'decoder.block.12.layer.0.SelfAttention.v.weight', 'decoder.block.3.layer.0.SelfAttention.k.weight', 'decoder.block.7.layer.1.EncDecAttention.v.weight', 'decoder.block.11.layer.1.EncDecAttention.q.weight', 'decoder.block.14.layer.0.SelfAttention.q.weight', 'decoder.block.0.layer.0.layer_norm.weight', 'decoder.block.1.layer.0.SelfAttention.k.weight', 'decoder.block.1.layer.0.SelfAttention.o.weight', 'decoder.block.2.layer.2.DenseReluDense.wo.weight', 'decoder.block.5.layer.1.EncDecAttention.v.weight', 'decoder.block.3.layer.0.SelfAttention.v.weight', '

# Precompute Text Feature

In [ ]:
# precompute features
from tqdm.notebook import tqdm
all_text_features = []
all_scores = []
for text, scores in tqdm(dataset):
    all_text_features.append(model.nlp([text])[0].cpu())
    all_scores.append(scores)


  0%|          | 0/7421 [00:00<?, ?it/s]

In [ ]:
precomputed_text_feats = torch.stack(all_text_features)
precomputed_scores = torch.stack(all_scores)
torch.save([precomputed_text_feats, precomputed_scores], "precomputed_t5_large_dataset.pt")

# Train with precomputed features

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl
from transformers import BertTokenizer, BertModel
from torchmetrics import ConfusionMatrix
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


NUM_CATEGORIES = 6
NUM_RUBRICS = 7
tokenizers = ["roberta-base", "bert-base-uncased", "t5-large"]
CURR_MODEL = "t5-large"

class PrecomputedTrainer(pl.LightningModule):
    def __init__(self):
        super().__init__()
        if CURR_MODEL == "t5-large":
            self.cls = nn.Sequential(
                nn.Linear(1024, 512),
                nn.ReLU(),
                nn.Linear(512, 250),
                nn.ReLU(),
                nn.Linear(250, NUM_CATEGORIES * NUM_RUBRICS)
            )
        else:
            self.cls = nn.Sequential(
                nn.Linear(768, 500),
                nn.ReLU(),
                nn.Linear(500, 250),
                nn.ReLU(),
                nn.Linear(250, NUM_CATEGORIES * NUM_RUBRICS)
            )

    def forward(self, text_feature):
#         print(text_feature.shape)
        return self.cls(text_feature)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
  
    
    def training_step(self, train_batch, batch_idx):
        text_feat, scores = train_batch
        logits = self.forward(text_feat).reshape(-1, NUM_CATEGORIES, NUM_RUBRICS) # B x 6 x 8
        loss = F.cross_entropy(logits, scores.long())
        
        self.log('train_ce_loss', loss, batch_size=len(text_feat))
        preds = logits.argmax(dim=1)
        self.log('train_acc', (preds == scores).float().mean(), batch_size=len(text_feat))
        return loss

    def validation_step(self, val_batch, batch_idx):
        text_feat, scores = val_batch
        logits = self.forward(text_feat).reshape(-1, NUM_CATEGORIES, NUM_RUBRICS) # B x 6 x 8
        
        loss = F.cross_entropy(logits, scores.long())
        
        self.log('val_ce_loss', loss, batch_size=len(text_feat))
        preds = logits.argmax(dim=1)
        self.log('val_acc', (preds == scores).float().mean(), batch_size=len(text_feat))
        self.log('val_adjacent', (torch.abs(preds - scores) <= 1).float().mean(), batch_size=len(text_feat))
        
        return {"scores": scores, "pred": preds}
    
    def validation_epoch_end(self, validation_step_outputs):
        scores_list = []
        prediction_list = []
        for step in validation_step_outputs:
            s = step["scores"]
            scores_list.append(s)
            p = step["pred"]
            prediction_list.append(p)
        scores = torch.cat(scores_list)
        prediction = torch.cat(prediction_list)
      
            
        scores_int = scores.int()
        prediction_int = prediction.int()
        
        prediction_int[prediction_int<1]=0
        prediction_int[prediction_int>5]=5
        
        
        for i in range(NUM_RUBRICS):
            confusion_matrix = ConfusionMatrix(num_classes=6)(prediction_int[:,i].cpu(), scores_int[:,i].cpu())
#             print(f"confusion matrix {i}", confusion_matrix.numpy())
            
            df_cm = pd.DataFrame(confusion_matrix.numpy(), index = range(6), columns=range(6))
            plt.figure(figsize = (10,7))
            fig_ = sns.heatmap(df_cm, annot=True, cmap='Blues').get_figure()
            plt.close(fig_)

            self.logger.experiment.add_figure(f"Confusion matrix {i}", fig_, self.current_epoch)
        
        
        acc_per_rubric_item = torch.mean((prediction == scores).float(), dim=0)
        ajc_acc_per_rubric_item = torch.mean((torch.abs(prediction - scores) <= 1).float(), dim=0)

        
        self.log('val_Claim_acc', acc_per_rubric_item[0])
        self.log('val_Reasons_acc', acc_per_rubric_item[1])
        self.log('val_Evidence_acc', acc_per_rubric_item[2])
        self.log('val_Reasoning_acc', acc_per_rubric_item[3])
        self.log('val_Organization_acc', acc_per_rubric_item[4])
        self.log('val_Audience_acc', acc_per_rubric_item[5])
        self.log('val_English_acc', acc_per_rubric_item[6])
        
        self.log('val_Claim_ajc_acc', ajc_acc_per_rubric_item[0])
        self.log('val_Reasons_ajc_acc', ajc_acc_per_rubric_item[1])
        self.log('val_Evidence_ajc_acc', ajc_acc_per_rubric_item[2])
        self.log('val_Reasoning_ajc_acc', ajc_acc_per_rubric_item[3])
        self.log('val_Organization_ajc_acc', ajc_acc_per_rubric_item[4])
        self.log('val_Audience_ajc_acc', ajc_acc_per_rubric_item[5])
        self.log('val_English_ajc_acc', ajc_acc_per_rubric_item[6])

# Start tensorboard

In [ ]:
# Download Ngrok to tunnel the tensorboard port to an external port
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
!./ngrok authtoken 26uW43ryj6Tp8oQY0YF6l0jauwt_3NdeYNLjBBXJPC4FB6TBa

# Run tensorboard as well as Ngrox (for tunneling as non-blocking processes)
import os
import multiprocessing


pool = multiprocessing.Pool(processes = 10)
results_of_processes = [pool.apply_async(os.system, args=(cmd, ), callback = None )
                        for cmd in [
                        f"tensorboard --logdir ./lightning_logs/ --host 0.0.0.0 --port 6006 &",
                        "./ngrok http 6006 &"
                        ]]

--2022-03-27 23:07:34--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 52.202.168.65, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  3.35MB/s    in 4.9s    

2022-03-27 23:07:40 (2.67 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# Get the url to access tensorboad
## Note: This block of code might throw an error while running all blocks at time. So run blocks one by one till the block below.
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://80c4-35-233-202-188.ngrok.io


2022-03-27 23:07:48.593521: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-27 23:07:48.673561: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-27 23:07:48.674524: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

TensorBoard 2.6.0 at http://0.0.0.0:6006/ (Press CTRL+C to quit)
W0327 23:07:53.299498 140537825629952 application.py:556] path /apple-touch-icon-

In [ ]:
# load precomputed dataset
precomputed_text_feats, precomputed_scores = torch.load("/kaggle/input/bert-precomputed/precomputed_t5_large_dataset.pt")
precomputed_text_feats, precomputed_scores = precomputed_text_feats.cuda(), precomputed_scores.cuda()
precomputed_dataset = torch.utils.data.TensorDataset(precomputed_text_feats, precomputed_scores)

In [ ]:
pl.utilities.seed.seed_everything(seed=1)
        
# data
dataset = precomputed_dataset
mnist_train, mnist_val = random_split(dataset, [int(len(dataset)*0.8), len(dataset)-int(len(dataset)*0.8)])

train_loader = DataLoader(mnist_train, batch_size=128, shuffle=True)
val_loader = DataLoader(mnist_val, batch_size=256, shuffle=False)

# model
model = PrecomputedTrainer()

checkpoint_callback = pl.callbacks.model_checkpoint.ModelCheckpoint(
    monitor="val_acc",
    dirpath="checkpoints/",
    filename="cls-all-data-t5_large-{epoch:02d}-{val_acc:.4f}",
    save_top_k=2,
    mode="max",
)

# training
trainer = pl.Trainer(gpus=1, precision=16, max_epochs=50,callbacks=[checkpoint_callback])
trainer.fit(model, train_loader, val_loader)
    


Validation sanity check: 0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:433: UserWarning: The number of training samples (47) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

W0327 23:11:54.136660 140537783666432 application.py:556] path /apple-touch-icon-precomposed.png not found, sending 404
W0327 23:11:54.138855 140537260648192 application.py:556] path /apple-touch-icon.png not found, sending 404
W0327 23:12:09.333014 140537783666432 application.py:556] path /apple-touch-icon.png not found, sending 404
W0327 23:12:09.336329 140537294219008 application.py:556] path /apple-touch-icon-precomposed.png not found, sending 404


In [ ]:
#given a model and text, evlauate scores 

def score(text, model):
    model = model.eval().cuda()
    return model(text)


In [ ]:
import warnings
warnings.filterwarnings('ignore')


In [ ]:
i = 24
t, s = mnist_val[i]
print(t)
print(s)

In [ ]:
loaded_model = Trainer.load_from_checkpoint('checkpoints/cerca-rubric-with-textlength-epoch=17-val_mse_loss=0.77.ckpt')

score([t], loaded_model)

In [ ]:
word_count, text_lengths, scores, scores_pred = [], [], [], []
from tqdm.notebook import tqdm
for t, s in tqdm(mnist_val):
    text_lengths.append(len(t))
    word_count.append(len(t.split()))
    scores.append(s[-1].item())
    scores_pred.append(score([t], loaded_model)[0][-1].item())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
plt.hist(text_lengths, bins=50)
pass

In [ ]:
#scatter plot of predicted final score vs log text_length for validation set
plt.scatter(np.log(text_lengths), scores_pred)

In [ ]:
ls "checkpoints/"

ls: cannot access 'checkpoints/': No such file or directory


In [ ]:
cp checkpoints/* .

W0327 23:12:59.387376 140537783666432 application.py:556] path /apple-touch-icon-precomposed.png not found, sending 404
W0327 23:12:59.390093 140537792059136 application.py:556] path /apple-touch-icon.png not found, sending 404
W0327 23:13:49.516059 140537294219008 application.py:556] path /apple-touch-icon.png not found, sending 404
W0327 23:13:49.519417 140537808844544 application.py:556] path /apple-touch-icon-precomposed.png not found, sending 404
W0327 23:13:51.255996 140537294219008 application.py:556] path /apple-touch-icon-precomposed.png not found, sending 404
W0327 23:13:51.260597 140537800451840 application.py:556] path /apple-touch-icon.png not found, sending 404


In [ ]:
rm -rf checkpoints/

In [ ]:
cp lightning_logs/version_0/* .

In [ ]:
rm -rf lightning_logs/

In [ ]:
rm *